# KODE OOK based on standard 7A- Receiver

#Rx side

In [5]:

## Hamming 16 bit -Receiver " Standard 7a"


# modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
# modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
# modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

# modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
modulated_signal = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]

# 1. OOK Demodulation
def ook_demodulation(modulated_signal):
    # Mengubah sinyal OOK kembali menjadi bit stream: 1 -> ON, 0 -> OFF
    return ''.join('1' if bit == 1 else '0' for bit in modulated_signal)

# 2. Remove Preamble
def remove_preamble(encoded_bits, preamble="10101010"):
    # Menghapus preamble yang ditambahkan sebelumnya
    return encoded_bits[len(preamble):]

# 3. RLL Decoding (Manchester)
def manchester_decoding(data_bits):
    decoded_bits = []
    # Setiap dua bit pada data adalah hasil Manchester encoding
    for i in range(0, len(data_bits), 2):
        if data_bits[i:i+2] == "10":
            decoded_bits.append('1')
        elif data_bits[i:i+2] == "01":
            decoded_bits.append('0')
    return ''.join(decoded_bits)

# 4. FEC Decoder (Hamming 8,4)
def hamming_decoder(encoded_bits):
    decoded_bits = []
    for i in range(0, len(encoded_bits), 8):
        # Ambil grup 8 bit yang sudah diterima
        group = encoded_bits[i:i+8]
        p1, p2, d1, p3, d2, d3, d4, p4 = group
        
        # Hitung syndrome untuk perbaikan bit error
        s1 = (int(d1) + int(d2) + int(d4) + int(p1)) % 2
        s2 = (int(d1) + int(d3) + int(d4) + int(p2)) % 2
        s3 = (int(d2) + int(d3) + int(d4) + int(p3)) % 2
        
        # Tentukan posisi bit yang error
        error_pos = s1 * 1 + s2 * 2 + s3 * 4
        
        if error_pos != 0:
            # Perbaiki bit yang error (posisi dihitung dari 1)
            error_pos -= 1  # Mengubah ke index 0
            group = list(group)
            group[error_pos] = '1' if group[error_pos] == '0' else '0'  # Membalikkan bit error

        # Ambil data yang terdekode (d1, d2, d3, d4)
        decoded_bits.append(group[2])
        decoded_bits.append(group[4])
        decoded_bits.append(group[5])
        decoded_bits.append(group[6])
    
    return ''.join(decoded_bits)

# Fungsi untuk mengubah biner menjadi karakter yang dapat dibaca
def binary_to_text(binary_string):
    # Pecah string biner menjadi blok 8-bit dan konversi setiap blok ke karakter
    n = 8  # Setiap karakter adalah 8-bit
    text = ''
    for i in range(0, len(binary_string), n):
        byte = binary_string[i:i+n]
        text += chr(int(byte, 2))  # Konversi byte biner ke karakter
    return text


# Main Receiver Function
def receiver(modulated_signal):
    # Step 1: OOK Demodulation
    demodulated_signal = ook_demodulation(modulated_signal)
    print(f"\nSignal Length: {len(demodulated_signal)}")
    print(f"Demodulated Signal: {demodulated_signal}")
    
    # Step 2: Remove Preamble
    signal_without_preamble = remove_preamble(demodulated_signal)
    print(f"Signal without Preamble: {signal_without_preamble}")
    
    # Step 3: RLL Decoding (Manchester)
    manchester_decoded_bits = manchester_decoding(signal_without_preamble)
    print(f"Manchester Decoded Bits: {manchester_decoded_bits}")
    
    # Step 4: FEC Decoding (Hamming 8,4)
    decoded_bits = hamming_decoder(manchester_decoded_bits)
    print(f"Decoded Bits (After Hamming): {decoded_bits}")
    

    # Convert decoded bits to readable text
    readable_text = binary_to_text(decoded_bits)
    print(f"Readable Text: {readable_text}")

    return decoded_bits

# Call the Receiver function
decoded_bits = receiver(modulated_signal)



Signal Length: 40
Demodulated Signal: 1010101010010101011010101001011010010110
Signal without Preamble: 10010101011010101001011010010110
Manchester Decoded Bits: 1000011110011001
Decoded Bits (After Hamming): 00110100
Readable Text: 4
